# In this project I will attempt to answer questions from a data set using a pre-trained BERT model


In [7]:
import torch
# Google API
import google

# BERT models
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

import speech_recognition as sr

# Google search API
from googlesearch import search
# Wikipedia API
import wikipedia

import re

# Google translate API
from gtts import gTTS

import os 


# BERT QA modelling
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


#### Speach recognition

In [8]:
def ask():
    # obtain audio from the microphone
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Ask a question!")
        audio = r.listen(source)

    # recognize speech using Google Speech Recognition
    try:
        # for testing purposes, we're just using the default API key
        # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
        # instead of `r.recognize_google(audio)`
        question =  r.recognize_google(audio)
        print("Google Speech Recognition thinks you said " + question)
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio.\n Would you like to type in your question or try speach again.\nIf you would like to try again press 'y'!")
        In = input()
        if In == y:
            ask()
        else:
            question = In
            return question
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
        print('Would you like to type in your question or try speach again.\nIf you would like to try again press y!')
        In = input()
        if In == y:
            ask()
        else:
            question = In
            return question
    return question

#### Google search

Asks Google the question and resives the URL responces

In [9]:

def goog(question):
    query = question

    Websites = []
    for j in search(query, tld="co.in", num=10, stop=10, pause=2): 
        Websites.append(j)
    return Websites


#### Wiki search

Extracts wikipedia urls and displace the summary of the most relevent page

In [10]:

def wiki(url):
    if len(re.findall('wikipedia.org', url)) >= 1:
        return url.rsplit('/', 1)[-1]
    else:
        return None
        
def wiki_serach(Websites):
    A=[]
    for url in Websites:
        wik = wiki(url)
        if wik != None:
            A.append(wik)
    return A


def Wiki(Websites):
    pages = wiki_serach(Websites)
    text = wikipedia.page(pages[0]).summary
    
    text = text.split(' ')
    Text = ''
    for word in text:
        Text.join(word)
        
    return Text



#### question answering

Uses the BERT model for question answering to find the answer in the text

In [11]:
#question = "Age of the universe"
    
def bert(text, question): 
    
    answer_text = text
    
    input_ids=tokenizer.encode(question,answer_text)
    # BERT only needs the token IDs, but for the purpose of inspecting the 
    # tokenizer's behavior, let's also get the token strings and display them.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)



    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

     #Run our example through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                     token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text


    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Combine the tokens in the answer and print it out.
    answer = ' '.join(tokens[answer_start:answer_end+1])

    print('Question: ' + question)
    print('Answer: "' + answer + '"')


    answer = re.sub(' ##', '', answer)
    print(answer)
    return answer

#### Answer to speach

In [12]:

def to_speach(answer):
    # The text that you want to convert to audio 
    mytext = answer

    # Language in which you want to convert 
    language = 'en-uk'

    # Passing the text and language to the engine,  
    # here we have marked slow=False. Which tells  
    # the module that the converted audio should  
    # have a high speed 
    myobj = gTTS(text=mytext, lang=language, slow=False) 

    # Saving the converted audio in a mp3 file named 
    # welcome  
    myobj.save("audio.mp3") 


    from pygame import mixer  # Load the popular external library

    mixer.init()
    mixer.music.load("audio.mp3")
    mixer.music.play() 
    mixer.music.stop()
    os.remove("audio.mp3")


In [25]:
def QA():
    question = ask()
    Websites = goog(question)
    print(Websites)
    text = Wiki(Websites)
    answer = bert(text, question)
    # Can answer with speach issue with files being open at the same time, will need fixing
    #to_speach(answer)
    
QA()

Ask a question!
Google Speech Recognition thinks you said how old is the universe
['https://www.scientificamerican.com/article/how-old-is-the-universe/?sa=X&ved=2ahUKEwjSifi7xfrsAhWyQUEAHUWqDB4Q9QF6BAgKEAI', 'https://www.scientificamerican.com/article/how-old-is-the-universe/', 'https://www.space.com/how-can-a-star-be-older-than-the-universe.html', 'https://www.space.com/24054-how-old-is-the-universe.html', 'https://en.wikipedia.org/wiki/Age_of_the_universe', 'https://en.wikipedia.org/wiki/Age_of_the_universe#Observational_limits', 'https://en.wikipedia.org/wiki/Age_of_the_universe#Cosmological_parameters', 'https://en.wikipedia.org/wiki/Age_of_the_universe#Planck', 'https://en.wikipedia.org/wiki/Age_of_the_universe#History', 'https://starchild.gsfc.nasa.gov/docs/StarChild/questions/question28.html']
Question: how old is the universe
Answer: "[SEP]"
[SEP]
